In [1]:
import pandas as pd
from os.path import join 
from core.data import Dataset
from core.configs import *
from core.train import Train
import optuna
import plotly.express as px
import mlflow
%cd ~/notebooks/petrol/
mlflow.set_tracking_uri('mlruns')

/home/prog3/notebooks/petrol


/home/prog3/.cache/pypoetry/virtualenvs/petrol-j8nzVRxz-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/prog3/.cache/pypoetry/virtualenvs/petrol-j8nzVRxz-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# 2022 - 1250

## Объединение с влагомером и водой

In [ ]:
df = Dataset(verbose=True)
df.load(join(DATA, 'raw', '2022', 'Режимный лист МУИС 2022.xlsx1250_clear.csv'), parse_dates=['Дата'])
df.drop(['Unnamed: 0'])

df.df.columns = [
 "Date","BlockP","BlockT",
 "C1P","C1T","C1L",
 "C2P","C2T","C2interfacial_L","С2petrol_L",

 "OH1T","OH1interfacial_L","OH1P",
 
 "C3T","C3L",
 "P1Tinput","P1Toutput","P1Pinput","P1Poutput","P1Тcoolant","P1Tgases","P1Рburner",
 "P2Tinput","P2Toutput","P2Pinput","P2Poutput","P2Tcoolant","P2Tgases","P2Pburner",
 "BEV1L","BEV1V_water","BEV1V_petrol","BEV1T_water"
]


vlagomer = pd.read_csv(join(DATA, 'raw', '2022', '1250_2022_vlagomer.csv'), parse_dates=['Дата']).drop(columns='Unnamed: 0')
vlagomer.columns = ['Date', 'Water_2hours', 'Vlagomer']

df.df = df.df.merge(vlagomer)
df.save(join(DATA, 'raw', '2022','2022_1250_joined_vlagomer.csv'))
df.df

## Подготовка к train

In [ ]:
cols_to_drop = ['BEV1L','BEV1V_water', 'BEV1V_petrol', 'BEV1T_water']
df_2022 = Dataset(verbose=False)
df_2022.load(join(DATA, 'preprocessed', '2022_1250_joined_vlagomer.csv'), parse_dates=['Date'])
df_2022.drop(cols_to_drop)
df_2022.astype(df_2022.cols[1:], 'float32')
df_2022.corr_matrix(df_2022.cols[1:], target='Vlagomer', show=True, height=400, width=1800, title='До скалирования/замены выбросов/minmax скалирования')
df_2022.scale(df_2022.cols[1:], method='standard')
df_2022.recovery_outliers(df_2022.cols[1:-1], save_interval=[-6,10], insert=False)
df_2022.scale(df_2022.cols[1:], method='minmax')


df_2022.corr_matrix(df_2022.cols[1:], target='Vlagomer', show=True, height=400, width=1800, title='До скалирования/замены выбросов/minmax скалирования')
# df_2022.time_series(df_2022.cols[1:], time_axis='Date', show=True, height=500, width=1750)
df_2022.save(join(DATA, 'preprocessed', '2022_1250_scaled.csv'))

### LightGBM

#### Все признаки:
    'BlockP', 'BlockT', 'C1P', 'C1T', 'C1L', 'C2P', 'C2T',
    'C2interfacial_L', 'С2petrol_L', 'OH1T', 'OH1interfacial_L', 'OH1P',
    'C3T', 'C3L', 'P1Tinput', 'P1Toutput', 'P1Pinput', 'P1Poutput',
    'P1Тcoolant', 'P1Tgases', 'P1Рburner', 'P2Tinput', 'P2Toutput',
    'P2Pinput', 'P2Poutput', 'P2Tcoolant', 'P2Tgases', 'P2Pburner', 'Water'

In [4]:
df_2022 = Dataset(verbose=False)
df_2022.load(join(DATA, 'preprocessed', '2022_1250_scaled.csv'), parse_dates=['Date'])

train = Train(df_2022, test_size=0.2)
train.optimize('lightgbm_2022_1250_scaled', 100) # дашборд запускается $ optuna-dashboard sqlite:///db.sqlite3
# train.try_all_trials(100)
train.train_with_trial(train.best_trials[0], 1000)
# train.plot_preds()

[I 2024-04-25 08:02:15,158] Using an existing study with name 'lightgbm_2022_1250_scaled' instead of creating a new one.
/home/prog3/notebooks/petrol/core/train.py:127: ExperimentalWarning:

set_metric_names is experimental (supported from v3.2.0). The interface can change in the future.

[I 2024-04-25 08:02:18,554] Trial 101 finished with values: {'r2': 0.7871857934374621, 'mae': 0.0558268530648705, 'mse': 0.006074042608847705, 'rmse': 0.07793614443150049} and parameters: {'n_estimators': 100, 'learning_rate': 0.03356523291133961, 'num_leaves': 502, 'subsample': 0.5254917251552156, 'colsample_bytree': 0.5374895199538191, 'min_data_in_leaf': 6}. 


dict_values([0.7871857934374621, 0.0558268530648705, 0.006074042608847705, 0.07793614443150049])


[I 2024-04-25 08:02:21,882] Trial 102 finished with values: {'r2': 0.7933605801681662, 'mae': 0.05543586519964213, 'mse': 0.00589780476124977, 'rmse': 0.0767971663621111} and parameters: {'n_estimators': 100, 'learning_rate': 0.02629812504286011, 'num_leaves': 1023, 'subsample': 0.9025525067820881, 'colsample_bytree': 0.5374895199538191, 'min_data_in_leaf': 6}. 


dict_values([0.7933605801681662, 0.05543586519964213, 0.00589780476124977, 0.0767971663621111])


[I 2024-04-25 08:02:22,686] Trial 103 finished with values: {'r2': 0.7870955051753254, 'mae': 0.056300329051413106, 'mse': 0.006076619573798286, 'rmse': 0.07795267521899608} and parameters: {'n_estimators': 100, 'learning_rate': 0.05639642448911333, 'num_leaves': 987, 'subsample': 0.656574900394001, 'colsample_bytree': 0.9421683489560035, 'min_data_in_leaf': 23}. 


...

dict_values([0.7829230439493263, 0.056278730106195836, 0.006195707992188417, 0.07871281974487013])


[I 2024-04-25 08:02:32,006] Trial 109 finished with values: {'r2': 0.7733278303236137, 'mae': 0.058540363390472874, 'mse': 0.006469570049355394, 'rmse': 0.08043363754894711} and parameters: {'n_estimators': 100, 'learning_rate': 0.08759491036745065, 'num_leaves': 711, 'subsample': 0.4934971082298534, 'colsample_bytree': 0.47108348022349145, 'min_data_in_leaf': 36}. 


dict_values([0.7733278303236137, 0.058540363390472874, 0.006469570049355394, 0.08043363754894711])


[I 2024-04-25 08:02:32,929] Trial 110 finished with values: {'r2': 0.7510625980474979, 'mae': 0.0630272898540198, 'mse': 0.007105053797012425, 'rmse': 0.08429148116513568} and parameters: {'n_estimators': 100, 'learning_rate': 0.1065800879858858, 'num_leaves': 807, 'subsample': 0.12769548437508427, 'colsample_bytree': 0.9400015606339096, 'min_data_in_leaf': 6}. 


dict_values([0.7510625980474979, 0.0630272898540198, 0.007105053797012425, 0.08429148116513568])


#### Признаки: 
       'BlockP', 'BlockT', 'C1P', 'C1T', 'C1L', 'P1Tinput',
       'P1Toutput', 'P1Pinput', 'P1Poutput', 'P1Тcoolant', 'P1Tgases',
       'P1Рburner', 'P2Tinput', 'P2Toutput', 'P2Pinput', 'P2Poutput',
       'P2Tcoolant', 'P2Tgases', 'P2Pburner'

In [9]:
df_2022.drop(['C2P', 'C2T',
       'C2interfacial_L', 'С2petrol_L', 'OH1T', 'OH1interfacial_L', 'OH1P',
       'C3T', 'C3L','Water'])

In [10]:
train = Train(df_2022)
train.optimize('lightgbm_2022_1250_scaled_до_печек_вкл', 100) # дашборд запускается $ optuna-dashboard sqlite:///db.sqlite3
train.try_all_trials(100)


[I 2024-04-23 09:31:39,214] A new study created in RDB with name: lightgbm_2022_1250_scaled_до_печек_вкл
/home/prog3/notebooks/petrol/core/train.py:139: ExperimentalWarning:

set_metric_names is experimental (supported from v3.2.0). The interface can change in the future.

[I 2024-04-23 09:31:39,720] Trial 0 finished with values: {'r2': 0.5263752008973497, 'mae': 0.09304645184303412, 'mse': 0.01351797540598422, 'rmse': 0.1162668284851024} and parameters: {'n_estimators': 100, 'learning_rate': 0.017239698233396755, 'num_leaves': 990, 'subsample': 0.7215340434373471, 'colsample_bytree': 0.12408605099935613, 'min_data_in_leaf': 95}. 
[I 2024-04-23 09:31:40,354] Trial 1 finished with values: {'r2': 0.7454703316850633, 'mae': 0.06285517705407244, 'mse': 0.007264665623281511, 'rmse': 0.08523300782725851} and parameters: {'n_estimators': 100, 'learning_rate': 0.07439305374435556, 'num_leaves': 476, 'subsample': 0.531346877504441, 'colsample_bytree': 0.7375066738524098, 'min_data_in_leaf': 57}

#### Признаки: 
    'BlockP', 'BlockT', 'C1P', 'C1T', 'C1L'


In [ ]:
df_2022.drop(['P1Tinput',
       'P1Toutput', 'P1Pinput', 'P1Poutput', 'P1Тcoolant', 'P1Tgases',
       'P1Рburner', 'P2Tinput', 'P2Toutput', 'P2Pinput', 'P2Poutput',
       'P2Tcoolant', 'P2Tgases', 'P2Pburner'])

In [ ]:
train = Train(df_2022)
train.optimize('lightgbm_2022_1250_блок_и_сепаратор', 100) # дашборд запускается $ optuna-dashboard sqlite:///db.sqlite3
train.try_all_trials(100)

### CatBoost

In [2]:
df_2022 = Dataset(verbose=False)
df_2022.load(join(DATA, 'preprocessed', '2022_1250_scaled.csv'), parse_dates=['Date'])

train = Train(df_2022,  model_name='catboost', metrics=['R2','MAE', 'RMSE'], directions=['maximize', 'minimize','minimize'], test_size=0.2)
train.optimize('catboost_2022_1250_scaled', 100) # дашборд запускается $ optuna-dashboard sqlite:///db.sqlite3
# train.try_all_trials(100)
train.train_with_trial(train.best_trials[0], 1000)
# train.plot_preds()

[I 2024-04-25 07:31:16,300] A new study created in RDB with name: catboost_2022_1250_scaled
/home/prog3/notebooks/petrol/core/train.py:127: ExperimentalWarning: set_metric_names is experimental (supported from v3.2.0). The interface can change in the future.
  study.set_metric_names(self.metrics_list)
Default metric period is 5 because MAE, R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2024-04-25 07:31:18,419] Trial 0 finished with values: {'R2': 0.715239162708251, 'MAE': 0.06918484272929039, 'RMSE': 0.09015270032191887} and parameters: {'n_estimators': 147, 'learning_rate': 0.024801370188845415, 'depth': 4, 'l2_leaf_reg': 4.949404289646627}. 


dict_values([0.715239162708251, 0.06918484272929039, 0.09015270032191887])


Default metric period is 5 because MAE, R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2024-04-25 07:31:20,342] Trial 1 finished with values: {'R2': 0.10777396803095773, 'MAE': 0.13105413238851407, 'RMSE': 0.1595791156838582} and parameters: {'n_estimators': 65, 'learning_rate': 0.001407475436630439, 'depth': 6, 'l2_leaf_reg': 1.359639900119248}. 


dict_values([0.10777396803095773, 0.13105413238851407, 0.1595791156838582])


Default metric period is 5 because MAE, R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2024-04-25 07:31:23,435] Trial 2 finished with values: {'R2': 0.7856016102008577, 'MAE': 0.05698472773691627, 'RMSE': 0.07822568388230752} and parameters: {'n_estimators': 68, 'learning_rate': 0.08569498903860184, 'depth': 12, 'l2_leaf_reg': 1.16124756637532}. 


...

dict_values([0.365300048296614, 0.11040494018640253, 0.1345931141030015])


Default metric period is 5 because MAE, R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2024-04-25 07:41:17,039] Trial 98 finished with values: {'R2': 0.34112577496228325, 'MAE': 0.11370586308308304, 'RMSE': 0.13713233411627834} and parameters: {'n_estimators': 160, 'learning_rate': 0.0035879626617611026, 'depth': 2, 'l2_leaf_reg': 4.899171255004723}. 


dict_values([0.34112577496228325, 0.11370586308308304, 0.13713233411627834])


Default metric period is 5 because MAE, R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2024-04-25 07:41:26,479] Trial 99 finished with values: {'R2': 0.635956196355826, 'MAE': 0.07988531624915236, 'RMSE': 0.10193314656926694} and parameters: {'n_estimators': 121, 'learning_rate': 0.009183066002866622, 'depth': 14, 'l2_leaf_reg': 3.149012616455851}. 
2024/04/25 07:41:26 INFO mlflow.tracking.fluent: Experiment with name 'catboost_2022_1250_scaled' does not exist. Creating a new experiment.


dict_values([0.635956196355826, 0.07988531624915236, 0.10193314656926694])
{'n_estimators': 1000, 'learning_rate': 0.08569498903860184, 'depth': 12, 'l2_leaf_reg': 1.16124756637532, 'custom_metric': ['R2', 'MAE', 'RMSE'], 'random_state': 2024, 'task_type': 'GPU', 'devices': '0-3'}
interval columns not set, guessing: ['BlockP', 'BlockT', 'C1P', 'C1T', 'C1L', 'C2P', 'C2T', 'C2interfacial_L', 'С2petrol_L', 'OH1T', 'OH1interfacial_L', 'OH1P', 'C3T', 'C3L', 'P1Tinput', 'P1Toutput', 'P1Pinput', 'P1Poutput', 'P1Тcoolant', 'P1Tgases', 'P1Рburner', 'P2Tinput', 'P2Toutput', 'P2Pinput', 'P2Poutput', 'P2Tcoolant', 'P2Tgases', 'P2Pburner', 'Water', 'Vlagomer']


In [3]:
train.train_with_trial(train.best_trials[0], 1000)

{'n_estimators': 1000, 'learning_rate': 0.08569498903860184, 'depth': 12, 'l2_leaf_reg': 1.16124756637532, 'custom_metric': ['R2', 'MAE', 'RMSE'], 'random_state': 2024, 'task_type': 'GPU', 'devices': '0-3'}
interval columns not set, guessing: ['BlockP', 'BlockT', 'C1P', 'C1T', 'C1L', 'C2P', 'C2T', 'C2interfacial_L', 'С2petrol_L', 'OH1T', 'OH1interfacial_L', 'OH1P', 'C3T', 'C3L', 'P1Tinput', 'P1Toutput', 'P1Pinput', 'P1Poutput', 'P1Тcoolant', 'P1Tgases', 'P1Рburner', 'P2Tinput', 'P2Toutput', 'P2Pinput', 'P2Poutput', 'P2Tcoolant', 'P2Tgases', 'P2Pburner', 'Water', 'Vlagomer']


# Обработка 2022 (2500) (data_2022.csv уже перезаписан)

In [ ]:
df = Dataset(verbose=True)
df.load(join(DATA, 'preprocessed', 'data_2022.csv'), parse_dates=['Дата'])
df.drop(['Unnamed: 0'])

# df.df[df.df['ОН-1/2,P отс. ,кгс/см²'].isin([0])].iloc[:,14:]

alerts = ['2022-05-17 10:00:00', '2022-06-06 08:00:00', '2022-07-28 14:00:00', '2022-12-25 12:00:00', '2022-09-03 16:00:00']
df.df[df.df['Дата'].isin(alerts)] # аномальные дни

df.df.drop(df.df[df.df['Дата'].isin(alerts)].index, inplace=True)

# df.time_series(df.cols[1:], time_axis='Дата', filepath=join(PLOTS, 'time_series', "2022_all"))
# df.corr_matrix(df.cols[1:], target='Добыча воды за 2 ч', filepath=join(PLOTS, 'correlation', "2022_all"), show=True)

# Идеи

In [ ]:
cols = df.cols[1:-2]
for col in cols:
    df.df[col] = df.zscore(df.df[col], window=300, thresh=3.0, coeff=1.0)
    # plot(df.df[col], df.df[col + '_cleaned'], max, min)


In [ ]:
cols = ['Water_2hours', 'Vlagomer']
for col in cols:
    df.df[col] = df.zscore(df.df[col], window=300, thresh=3.0, coeff=1.0)
    # plot(df.df[col], df.df[col + '_cleaned'], max, min)
df.corr_matrix(df.cols[1:], target='Vlagomer', show=True)